In [1]:
## all imports
from IPython.display import HTML
import chardet
import io
import numpy as np
np.seterr(divide='ignore', invalid='ignore')
import os
import urllib.request
from urllib.request import urlopen
from bs4 import BeautifulSoup
from time import sleep
import requests
import operator
import socket
import _pickle
import math
import re # regular expressions

from pandas import Series
import pandas as pd
from pandas import DataFrame

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = (20, 10)
import json

import seaborn as sns
sns.set_context("talk")
sns.set_style("white")

from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import time
from datetime import datetime

#from secret import *

### FUNCTION TO CREATE WIKIPEDIA YEAR URL

In [2]:
def find_url(row):
    Base_Link_01= "https://en.wikipedia.org/wiki/"
    Base_Link_02= "_AFL_season"
    Variable= row
    link = Base_Link_01 + Variable + Base_Link_02
    return (link)

### FUNCTION: ENSURE ALL MONTH NAMES ARE CONSISTENT (REMOVE SHORTENING)

In [3]:
def correct_month (x):
    first_space_index = x.index(" ")
    day = x[:first_space_index]
    #print (day)
    time = x[first_space_index:].strip()
    space_index = time.index(" ")
    hours = time[space_index:].strip()
    split_month = x.split(" ")[1]
    #print (split_month)
    
    months_tocheck = ["January", "Febuary ", "March", "April", "May", "June", "July", "August", "September", "October", "Novemeber", "December"]
    
    #if any (split_month for month in months_tocheck):
    if split_month in months_tocheck:
        #print ("it is a correct month")
        return (x)
    
       
    else:
        month_dict = {
        "Mar" : "March",
        "Apr" : "April",
        "May" : "May",
        "June" : "June",
        "July" : "July",
        "Aug" : "August",
        "Sep" : "September",
        "Oct" : "October",
        }
        #print (month_dict[split_month])
        #print ("is not a correct minth")
        return (day + " " + month_dict[split_month] + " " + hours)


        


### FUNCTION: RETURN GAMES DATE

In [4]:
def the_date (x):
    first_space_index = x.index(" ")
    day = x[:first_space_index]
    return (day)

### FUNCTION: RETURN GAMES MONTH

In [5]:
def the_month (x):
    first_space_index = x.index(" ")
    day = x[:first_space_index]
    #print (day)
    time = x[first_space_index:].strip()
    space_index = time.index(" ")
    hours = time[space_index:].strip()
    split_month = x.split(" ")[1]
    return (split_month)
    #print (split_month)

### FUNCTION: RETURN TIME IN HOURS- PERHAPS CONVERT TO 24 HOUR TIME?

In [6]:
def the_time (x):
    first_space_index = x.index(" ")
    day = x[:first_space_index]
    #print (day)
    time = x[first_space_index:].strip()
    space_index = time.index(" ")
    hours = time[space_index:].strip()
    split_month = x.split(" ")[1]
    return (hours[1:-1])
    #print (split_month)

### FUNCTION: RETURN MATCH DAY IF IT EXISTS

In [7]:
def if_dayexists (x):
    ## we know that if it has a day it will have a comma
    #x = "Thursday, 14 June (5:40 pm)"
    count = x.count(",")
    if count >0:
        date = x.split(",")[0].strip()
        return (date)
    else: 
        date = x
        return ("No Day Data Provided")
    


### FUNCTION: TO REMOVE THE DAY FROM THE DATE

In [8]:
def day_conversion (x):
    ## we know if it has a day it will have a comma
    #x = "Thursday, 14 June (5:40 pm)"
    count = x.count(",")
    if count >0:
        date = x.split(",")[1].strip()
        #print (date)
        #first_colon_index = x.index(",")
        #print (first_colon_index)
        #date = x[first_colon_index+1:].strip()
        return (date)
    else: 
        date = x
        return (date)
    
    

### FUNCTION: TO REMOVE ANY WORDS BEFORE THE DATE (EG. PF1:)

In [9]:
def date_conversion (x):
    #x = "PF1: Friday, 25 September (6:20 pm)"
    #x = "Thursday, 14 June (5:40 pm)"
    count = x.count(":")
    if count >1:
        first_colon_index = x.index(":")
        date = x[first_colon_index+1:].strip()
        return (date)
    else: 
        date = x
        return (date)

### FUNCTION: TO CREATE PANDAS STRIPTIME + DATETIME

In [131]:
def structured_times_A (x):
    # pass in x, list how it is ordered
    struct_time = time.strptime(x, "%Y %d %B (%I:%M %p)")
    dt = datetime.fromtimestamp(time.mktime(struct_time))
    return (dt)

In [142]:
def structured_times_B (x):
    # pass in x, list how it is ordered
    struct_time = time.strptime(x, "%Y %d %B")
    dt = datetime.fromtimestamp(time.mktime(struct_time))
    return (dt)

In [ ]:
2004 26 March

### FUNCTION: MATCH SCORES DICTIONARY (SINGLE LINE DATA)

In [11]:
def scores_dictionary (test):
    
    short_home_score = (''.join(text for text in test[1].find_all(text=True) if text.parent.name != "a")).strip()
    home_goals= short_home_score.split('.')[0]
    home_behinds= short_home_score.split('.')[1].split('(')[0]
    home_total = (short_home_score.split('.')[1].split('(')[1])[:-1]
    short_away_score = (''.join(text for text in test[3].find_all(text=True) if text.parent.name != "a")).strip()
    away_goals= short_away_score.split('.')[0]
    away_behinds= short_away_score.split('.')[1].split('(')[0]
    away_total = (short_away_score.split('.')[1].split('(')[1])[:-1]
    
    
    ind_match_dict= {
         "Home Goals": home_goals,
         "Home Behinds": home_behinds,
         "Home Total": home_total,
         "Away Goals": away_goals,
         "Away Behinds": away_behinds,
         "Away Total": away_total}

    return (ind_match_dict)

### FUNCTION: MATCH SCORES DICTIONARY (MULTI LINE DATA- GRAND FINALS)

In [12]:
def scores_dictionary_02 (match_goals):
    

    test_data = match_goals.find_all('td')
    home_score = test_data[1].find('span').string
    home_total =  (home_score.split(" ")[1][1:-1])
    home_goals = (home_score.split(" ")[0]).split(".")[0]
    home_behinds = (home_score.split(" ")[0]).split(".")[1]
    away_score = test_data[3].find('span').string
    away_total =  (away_score.split(" ")[1][1:-1])
    away_goals = (away_score.split(" ")[0]).split(".")[0]
    away_behinds = (away_score.split(" ")[0]).split(".")[1]

    
    ind_match_dict= {
         "Home Goals": home_goals,
         "Home Behinds": home_behinds,
         "Home Total": home_total,
         "Away Goals": away_goals,
         "Away Behinds": away_behinds,
         "Away Total": away_total}

    return (ind_match_dict)

### FUNCTION: TYPE A- GENERAL MATCH INFO  DICTIONARY

In [134]:
def match_dictionary_A(test,this_round, this_year):

    # FUNCTION TAKE OFF FRONT PREFIX
    time_date= date_conversion(test[0].string)

    ### FUNCTION TO ASSIGN DAY IF IT EXISTS (BEFORE WE STRIP IT OFF FOR CLARITY AS NOT NEEDED)
    day = if_dayexists (time_date)

    ### FUNCTION TO TAKE OFF THE DAY AT THE START (AS NOT CONSISTENT)
    convert_time = day_conversion(time_date)


    ### FUNCTION TO ENSURE CONSISTENT MONTH SYNTAX
    correct_time_syn = correct_month (convert_time)

    #print (correct_time_syn)
    ### FUNCTION TO ASSIGN DATE, MONTH AND TIME
    date = the_date(correct_time_syn)
    month = the_month(correct_time_syn)


    ### ADDS THE CURRENT YEAR
    convert_time_year = (str(this_year)+ " " + convert_time)
    
 

    ### FUNCTION THAT CONVERTS TO TIME STRUCTURE
    struct_time = structured_times_A(convert_time_year)
    #print (struct_time)

    home_team = test[1].find('a').string
    away_team = test[3].find('a').string
    venue = test[4].find('a').string
    venue_crowd = (''.join(text for text in test[4].find_all(text=True) if text.parent.name != "a")).strip()
    venue_crowd_= (venue_crowd.split(':')[1])[:-1].strip()
    final_crowd = venue_crowd_.replace(",", "")
    
    try: 
        match_report_url = test[5].find('a')['href']
        
    except:
        match_report_url = "no match report"
    

    match_dict= {
     "Day": day,
     "Date": date,
     "Month": month,
     "Year": this_year,
     "Round": this_round,         
     "Time": time,
     "Structured Data": struct_time,
     "Home Team": home_team,
     "Away Team": away_team,
     "Venue": venue,
     "Crowd": final_crowd,
     "Match Link": match_report_url}
    
    return (match_dict)

### FUNCTION: TYPE B- GENERAL MATCH INFO  DICTIONARY

In [196]:
def match_dictionary_B(test, this_round, this_year):


    time_date= date_conversion(test[6].string)

    day = if_dayexists (time_date)

    convert_time = day_conversion(time_date)

    convert_time_year = (str(this_year)+ " " + convert_time)
    
    
    date = convert_time.split(" ")[0]
    month = convert_time.split(" ")[1]
    struct_time = structured_times_B(convert_time_year)
    home_team = test[0].find('a').string
    try:
        home_score = test[1].find('b').string
        
    except:
        home_score = test[1].string
        
    home_goals = "this"
    home_behinds = "this"
    home_total = "this"    

    away_team = test[2].find('a').string
    try:
        away_score = test[3].find('b').string
        
    except:
        away_score = test[3].string
        
    away_goals = "this"
    away_behinds = "this"
    away_total = "this"
    venue = test[4].find('a').string
    final_crowd = test[5].string
    match_report_url = "No match report"
    time = "No time provided"
    
    
    #print (day, date, month, this_year, this_round, time, struct_time, home_team, away_team, venue, final_crowd)
    #print (match_report_url, home_goals, home_behinds, home_total, away_goals, away_behinds, away_total)
    

    match_dict_= {
     "Day": day,
     "Date": date,
     "Month": month,
     "Year": this_year,
     "Round": this_round,         
     "Time": time,
     "Structured Data": struct_time,
     "Home Team": home_team,
     "Away Team": away_team,
     "Venue": venue,
     "Crowd": final_crowd,
     "Match Link": match_report_url,
     "Home Goals": home_goals,
     "Home Behinds": home_behinds,
     "Home Total": home_total,
     "Away Goals": away_goals,
     "Away Behinds": away_behinds,
     "Away Total": away_total}
    
    #print (match_dict)
    return (match_dict_)




### FUNCTION: MATCHDAY FOR LARGE TABLE TYPE

In [112]:
def match_function_A(match, this_round, this_year):


    try:

        namestocheck_good = ["Round", "Qualifying ", "Elimination", "Semi", "Preliminary", "Grand"]
        if any (name in this_round for name in namestocheck_good):
            if this_round == "Grand Final" or this_round == "Grand Final Replay":
                match_data = match[0]
            else:
                match_data = match

                 
  
            test = match_data.find_all('td')


            ### check if a score is return 
            test_for_score = (''.join(text for text in test[1].find_all(text=True) if text.parent.name != "a")).strip()
            if test_for_score:
                standard_dict = match_dictionary_A(test,this_round, this_year)
                #print (standard_dict)
                scores_dict = scores_dictionary(test)
                #print (scores_dict)
                dictionary = {**standard_dict, **scores_dict}
                return (dictionary)
            
            else: 

                standard_dict = match_dictionary_A(test,this_round, this_year)

                scores_dict = scores_dictionary_02(match[1])
                dictionary = {**standard_dict, **scores_dict}
                return (dictionary)

        
        else:
            print ("no name in scope")

    
    except Exception as e:
        print("this row does not have match data", e)


### FUNCTION: MATCHDAY FOR B- SMALL TABLE

In [198]:
def match_function_B(match,this_year):
    
    test = match.find_all('td')
    
    if len(test) > 7:
        this_round = test[0].string
        test = test[1:]
        dictionary = match_dictionary_B(test, this_round, this_year)
        print (dictionary)
        return (dictionary)
 


    else:
        test = test
        this_round = "Not Defined"
        dictionary = match_dictionary_B(test, this_round, this_year)
        print (dictionary)

    


## Why are we not returning this dictioopnary property?
## maybe we could indent it?


### FUNCTION: CREATE DATA  FOR EACH ROUND (LIST TYPE A)

In [147]:
def round_function_A(round_number, year_data,current_year):    
    games= year_data[round_number].find_all('tr')
    ## Find the round header
    round_head = (''.join(text for text in games[1].find_all(text=True) if text.parent.name != "a")).strip()
    #print (round_head)
    #if isinstance(round_head,str) == False:
    ## Check is string is empty- empty strings are falsy- so we are asking if not false- double negative- not false = true
    if not round_head:
        round_head = games[1].find('a').string

    this_round = []
    
 
    if round_head == "Grand Final" or round_head == "Grand Final Replay":
        result= match_function_A(games[2:4],round_head, current_year)
        if result:
            this_round.append(result)
        return (this_round)

    else:  
        round_matchday = games[2:-1]
        for match in round_matchday:
            result= match_function_A(match, round_head, current_year)
            if result:
                this_round.append(result)
        return (this_round)


### FUNCTION: CREATE DATA  FOR EACH ROUND (LIST TYPE B)

In [157]:
def round_function_B(round_number, year_data,current_year):    
    games= year_data[round_number].find_all('tr')

    this_round = []
    
    round_matchday = games[1:]

    
    for match in round_matchday:
        result= match_function_B(match,current_year)
        if result:
            this_round.append(result)
    return (this_round)

## TRY BOTH TABLE TYPES THROUGH EVERY YEAR (2001-2017)

### FUNCTION: LARGE TABLE- A

In [20]:
def table_typeA (year):    
    current_year = str(year)
    url = find_url(current_year)
    req = requests.get(url)
    soup = BeautifulSoup(req.text, 'html.parser')

    year_data = soup.find("div", {"id": "mw-content-text"}).find_all('table', {"style": "width: 100%; background-color: #f1f5fc; border: 2px solid #D0E5F5"})
    return (year_data)

### FUNCTION: SMALL TABLE- B

In [21]:
def table_typeB (year):    
    current_year = str(year)
    url = find_url(current_year)
    req = requests.get(url)
    soup = BeautifulSoup(req.text, 'html.parser')

    year_data= soup.find("div", {"id": "mw-content-text"}).find_all("table", {"class": "wikitable succession-box"})
    return (year_data)

### FUNCTION: CREATE DATA FOR EACH YEAR + TURN INTO 'MEGALIST'

In [199]:
mega_list = []

# 2002 is a mix
## 22004 only old
test_year = 2004

for year in list(range(test_year,test_year + 1)):
#for year in list(range(2000,2018,1)):

        ## Big Table
        year_data_A = table_typeA (year)
        for current_round in range(len(year_data_A)):

            this_round = round_function_A(current_round, year_data_A, year)
            mega_list.extend(this_round)
            
            
        ## Small Table  
        year_data_B = table_typeB (year)
        for current_round in range(len(year_data_B)):
            this_round = round_function_B(current_round, year_data_B, year)
            
    
print (mega_list)


{'Day': 'Friday', 'Date': '26', 'Month': 'March', 'Year': 2004, 'Round': 'Not Defined', 'Time': 'No time provided', 'Structured Data': datetime.datetime(2004, 3, 26, 0, 0), 'Home Team': 'Richmond', 'Away Team': 'Collingwood', 'Venue': 'MCG', 'Crowd': '55,622', 'Match Link': 'No match report', 'Home Goals': 'this', 'Home Behinds': 'this', 'Home Total': 'this', 'Away Goals': 'this', 'Away Behinds': 'this', 'Away Total': 'this'}
{'Day': 'Saturday', 'Date': '27', 'Month': 'March', 'Year': 2004, 'Round': 'Not Defined', 'Time': 'No time provided', 'Structured Data': datetime.datetime(2004, 3, 27, 0, 0), 'Home Team': 'Fremantle', 'Away Team': 'Carlton', 'Venue': 'Subiaco Oval', 'Crowd': '34,177', 'Match Link': 'No match report', 'Home Goals': 'this', 'Home Behinds': 'this', 'Home Total': 'this', 'Away Goals': 'this', 'Away Behinds': 'this', 'Away Total': 'this'}
{'Day': 'Saturday', 'Date': '27', 'Month': 'March', 'Year': 2004, 'Round': 'Not Defined', 'Time': 'No time provided', 'Structured Da

In [186]:
df = pd.DataFrame(mega_list)
#df.head(100)
df.shape

(207, 18)

### BREAK

### TURN MEGALIST INTO PANDAS DATAFRAME + REVIEW

In [48]:
df = pd.DataFrame(mega_list)
#df.head(400)
df.shape

(1514, 17)

In [126]:
df.loc[df['Round'] == 'Grand Final']

,Away Behinds,Away Goals,Away Team,Away Total,Crowd,Date,Day,Home Behinds,Home Goals,Home Team,Home Total,Match Link,Month,Round,Structured Data,Time,Venue,Year
184,8,10,St Kilda,68,100016,25,No Day Data Provided,14,9,Collingwood,68,http://www.afl.com.au/news/newsarticle/tabid/2...,September,Grand Final,2010-09-25 14:30:00,2:30 pm,MCG,2010
381,11,18,Geelong,119,99537,1,Saturday,9,12,Collingwood,81,http://www.afl.com.au/news/newsarticle/tabid/2...,October,Grand Final,2011-10-01 14:30:00,2:30 pm,MCG,2011
588,7,14,Sydney,91,99683,29,Saturday,15,11,Hawthorn,81,http://www.afl.com.au/match-centre/2012/27/haw...,September,Grand Final,2012-09-29 14:30:00,2:30 pm,MCG,2012
795,14,8,Fremantle,62,100007,28,Saturday,11,11,Hawthorn,77,http://www.afl.com.au/match-centre/2013/27/haw...,September,Grand Final,2013-09-28 14:30:00,2:30 pm,MCG,2013
1002,11,21,Hawthorn,137,99460,27,Saturday,8,11,Sydney,74,http://www.afl.com.au/match-centre/2014/27/syd...,September,Grand Final,2014-09-27 14:30:00,2:30 pm,MCG,2014
1208,13,8,West Coast,61,98633,3,Saturday,11,16,Hawthorn,107,http://www.afl.com.au/match-centre/2015/27/haw...,October,Grand Final,2015-10-03 14:30:00,2:30 pm,MCG,2015
1415,11,13,Western Bulldogs,89,99981,1,Saturday,7,10,Sydney,67,http://www.afl.com.au/match-centre/2016/27/syd...,October,Grand Final,2016-10-01 14:30:00,2:30 pm,MCG,2016


### IDENTIFY UNIQUE VENUE TYPES

In [20]:
df.Venue.unique()

array(['MCG', 'The Gabba', 'Etihad Stadium', 'AAMI Stadium',
       'Subiaco Oval', 'Gabba', 'ANZ Stadium', 'SCG', 'Skilled Stadium',
       'Aurora Stadium', 'Manuka Oval', 'TIO Stadium', 'Patersons Stadium',
       'Metricon Stadium', "Cazaly's Stadium", 'Adelaide Oval',
       'Blundstone Arena', 'Blacktown ISP', 'Simonds Stadium',
       'Škoda Stadium', 'StarTrack Oval Canberra', 'Westpac Stadium',
       'Spotless Stadium', 'StarTrack Oval', 'TIO Traeger Park',
       'Domain Stadium', 'UNSW Canberra Oval',
       'University of Tasmania Stadium', 'Jiangwan Stadium'], dtype=object)

### CREATE CONSISTENT VENUE TYPES (NO DOUBLE UPS)

In [21]:
def rationalise_venue(row):
    di_cities= {"MCG": "MCG",
     "The Gabba": "Gabba",
     "Gabba": "Gabba",
     "Etihad Stadium": "Etihad Stadium",
     "Adelaide Oval": "Adelaide Oval",
     "Manuka Oval": "Manuka Oval",
     "UNSW Canberra Oval": "Manuka Oval",
     "StarTrack Oval": "Manuka Oval",
     "StarTrack Oval Canberra": "Manuka Oval",
     "Domain Stadium": "Domain Stadium",
     "Subiaco Oval": "Domain Stadium",
     "Patersons Stadium": "Domain Stadium",
     "Metricon Stadium": "Metricon Stadium",
     "Blundstone Arena": "Blundstone Arena",
     "SCG": "SCG",
     "Spotless Stadium": "Spotless Stadium",
     "Blacktown ISP": "Spotless Stadium",
     "Škoda Stadium": "Spotless Stadium",
     "ANZ Stadium": "ANZ Stadium",
     "Simonds Stadium": "Simonds Stadium",
     "Skilled Stadium": "Simonds Stadium",
     "Aurora Stadium": "University of Tasmania Stadium",
     "University of Tasmania Stadium": "University of Tasmania Stadium",
     "Cazaly's Stadium": "Cazaly's Stadium",
     "TIO Traeger Park": "TIO Traeger Park",
     "TIO Stadium": "TIO Stadium",    
     "Westpac Stadium": "Westpac Stadium",
     "AAMI Stadium": "AAMI Stadium",
     "Jiangwan Stadium": "Jiangwan Stadium",
     "Westpac Stadium": "Westpac Stadium"}
    
    return di_cities[row.Venue]

In [58]:
df["Venue"] = df.apply(rationalise_venue, axis=1)
df.Venue.unique()

array(['MCG', 'Gabba', 'Etihad Stadium', 'AAMI Stadium', 'Domain Stadium',
       'ANZ Stadium', 'SCG', 'Simonds Stadium',
       'University of Tasmania Stadium', 'Manuka Oval', 'TIO Stadium',
       'Metricon Stadium', "Cazaly's Stadium", 'Adelaide Oval',
       'Blundstone Arena', 'Spotless Stadium', 'Westpac Stadium',
       'TIO Traeger Park', 'Jiangwan Stadium'], dtype=object)

### CREATE CITIES LIST BASED ON VENUES

In [24]:
def get_city(row):
    di_cities= {"MCG": "Melbourne",
     "Gabba": "Brisbane",
     "Etihad Stadium": "Melbourne",
     "Adelaide Oval": "Adelaide",
     "Manuka Oval": "Canberra",
     "Domain Stadium": "Perth",
     "Metricon Stadium": "Gold Coast",
     "Blundstone Arena": "Hobart",
     "SCG": "Sydney",
     "Spotless Stadium": "Western Sydney",
     "ANZ Stadium": "Western Sydney",
     "Simonds Stadium": "Geelong",
     "University of Tasmania Stadium": "Launceston",
     "Cazaly's Stadium": "Cairns",
     "TIO Traeger Park": "Alice Springs",
     "TIO Stadium": "Darwin",    
     "Westpac Stadium": "Wellington",
     "AAMI Stadium": "Adelaide",
     "Jiangwan Stadium": "Shanghai"}
                
    return di_cities[row.Venue]

In [59]:
df["City"] = df.apply(get_city, axis=1)

### CREATE WEATHER STATION ID'S LIST BASED ON CITIES

In [33]:
def get_station_ID(row):
    di_stations= {
    "Melbourne": "1487214643&p_stn_num=086232",
     "Brisbane": "334797792&p_stn_num=040913",
     "Adelaide": "105924303&p_stn_num=023011",
     "Canberra": "986923182&p_stn_num=070246",
     "Perth": "16771239&p_stn_num=009151",         
     "Gold Coast": "326713690&p_stn_num=040416",
     "Hobart": "1768351258&p_stn_num=094030",
     "Sydney": "873151344&p_stn_num=066073",
     "Western Sydney": "871566312&p_stn_num=066013",
     "Geelong": "1519465927&p_stn_num=087162",
     "Launceston": "1658844915&p_stn_num=091072",
     "Cairns": "194935783&p_stn_num=031218",
     "Alice Springs": "48632698&p_stn_num=015590",
     "Darwin": "40163856&p_stn_num=014167",    
     "Wellington": "N/A",
     "Shanghai": "N/A"}
                
    return di_stations[row.City]


### copy city links from below

In [36]:
df["station_ID"] = df.apply(get_station_ID, axis=1)

In [ ]:
df["station_ID"] = df.apply(get_station_ID, axis=1)
df.head(2)

### SELECT ROW BASED ON COLUMN VALUE (EG.VENUE IS METRICON)

In [60]:
df.loc[df['Venue'] == 'Metricon Stadium']

,Away Behinds,Away Goals,Away Team,Away Total,Crowd,Date,Day,Home Behinds,Home Goals,Home Team,Home Total,Match Link,Month,Round,Structured Data,Time,Venue,Year,City
275,13,21,Geelong,139,21485,28,Saturday,13,10,Gold Coast,73,http://www.afl.com.au/news/newsarticle/tabid/2...,May,Round 10,2011-05-28 19:20:00,7:20 pm,Metricon Stadium,2011,Gold Coast
290,14,18,North Melbourne,122,14945,11,Saturday,9,9,Gold Coast,63,http://www.afl.com.au/news/newsarticle/tabid/2...,June,Round 12 (Women's Round & Queen's Birthday),2011-06-11 19:10:00,7:10 pm,Metricon Stadium,2011,Gold Coast
305,8,17,Western Bulldogs,110,16149,25,Saturday,10,13,Gold Coast,88,http://www.afl.com.au/news/newsarticle/tabid/2...,June,Round 14,2011-06-25 14:10:00,2:10 pm,Metricon Stadium,2011,Gold Coast
322,16,15,Sydney,106,16488,9,Saturday,12,4,Gold Coast,36,http://www.afl.com.au/news/newsarticle/tabid/2...,July,Round 16,2011-07-09 19:10:00,7:10 pm,Metricon Stadium,2011,Gold Coast
338,15,19,Collingwood,129,23302,23,Saturday,9,11,Gold Coast,75,http://www.afl.com.au/news/newsarticle/tabid/2...,July,Round 18,2011-07-23 19:10:00,7:10 pm,Metricon Stadium,2011,Gold Coast
346,14,10,St Kilda,74,17482,30,Saturday,18,6,Gold Coast,54,http://www.afl.com.au/news/newsarticle/tabid/2...,July,Round 19,2011-07-30 19:10:00,7:10 pm,Metricon Stadium,2011,Gold Coast
367,17,18,Adelaide,125,16168,20,Saturday,10,9,Gold Coast,64,http://www.afl.com.au/news/newsarticle/tabid/2...,August,Round 22,2011-08-20 14:10:00,2:10 pm,Metricon Stadium,2011,Gold Coast
383,10,16,Hawthorn,106,19314,3,Saturday,13,14,Gold Coast,97,http://www.afl.com.au/news/newsarticle/tabid/2...,September,Round 24,2011-09-03 13:10:00,1:10 pm,Metricon Stadium,2011,Gold Coast
403,23,19,Adelaide,137,12790,31,Saturday,8,10,Gold Coast,68,http://www.afl.com.au/match-centre/2012/1/gcfc...,March,Round 1,2012-03-31 15:45:00,3:45 pm,Metricon Stadium,2012,Gold Coast
422,15,15,Essendon,105,17069,14,Saturday,10,13,Gold Coast,88,http://www.afl.com.au/match-centre/2012/3/gcfc...,April,Round 3,2012-04-14 19:40:00,7:40 pm,Metricon Stadium,2012,Gold Coast


### EXPORT DATA TO CSV

In [143]:
df.to_csv("test.csv")